In [1]:
from pyproj import CRS
import geopandas as gpd
import pandas as pd
import folium
import requests
import geojson

print('Fetching data...')
# URL to the WFS server
url = 'https://paituli.csc.fi/geoserver/paituli/wfs?'

# Specify parameters (read data in json format). 
params = dict(service='wfs', 
              version='2.0.0', 
              request='GetFeature', 
              typeName='paituli:tike_paavo_2021', 
              outputFormat='json')

# Fetch data from WFS using requests
r = requests.get(url, params=params)
    
# Create GeoDataFrame from geojson
data = gpd.GeoDataFrame.from_features(geojson.loads(r.content))

munici_fp = 'data/alueryhmittely_posnro_2021_fi.csv'
munici_codes = pd.read_csv(munici_fp,encoding='utf-8',
                   delimiter =';',
                   header=2,
                   usecols=['Postinumeroalueen nimi','Kunnan nimi','Seutukunta','Maakunnan nimi']
                  )

# Add description of crs from 3067, merge two datasets, and reproject to Web Mercator     
data.crs = CRS.from_epsg(3067)
data_merged = data.merge(munici_codes, left_on='nimi',right_on='Postinumeroalueen nimi')
data_merged = data_merged.to_crs(3857)
print('Data fetched!')

Fetching data...
Data fetched!


In [2]:
# Use only needed columns
geo_series = data_merged[['postinumer','nimi','Kunnan nimi','Maakunnan nimi','pinta_ala','he_vakiy','te_as_valj','geometry']]

# Create map instance
m = folium.Map(location = [64,27],
               zoom_start=5,
               tiles='CartoDBdark_matter',
               control_scale=True,
               max_bounds=True
              )
# Add features
folium.features.GeoJson(geo_series,
                        tooltip=folium.features.GeoJsonTooltip(fields=['postinumer','nimi','Kunnan nimi','Maakunnan nimi','pinta_ala','he_vakiy','te_as_valj'],
                                                                aliases=['Postal code', 'Postal area name','Municipality',
                                                                         'Province','Area(km²)','Population',
                                                                         'Occupancy rate (floor area per resident)'],
                                                                labels=True,
                                                                sticky=True
                                                                            )
                       ).add_to(m)
    
folium.LayerControl().add_to(m)
m.save('Postal_areas.html')